# Database Test Notebook

In [ ]:
%pip install -q mysql-connector-python psycopg2-binary

### References

- https://www.geeksforgeeks.org/how-to-connect-python-with-sql-database/#
- https://www.datacamp.com/tutorial/tutorial-postgresql-python
- https://stackoverflow.com/questions/12047193/how-to-convert-sql-query-result-to-pandas-data-structure

In [ ]:
import os
import mysql.connector
import psycopg2
import pandas as pd


## Connect to the Database

In [ ]:
# Creating connection object
mydb = mysql.connector.connect(
    database = os.environ['DB_NAME'],
    host = os.environ['DB_HOST'],
    user = os.environ['DB_USER'],
    password = os.environ['DB_PASSWORD']
)

In [ ]:
# Creating an instance of 'cursor' class which is used to execute the 'SQL' statements in 'Python'
cursor = mydb.cursor()

In [ ]:
query = "SELECT distinct(meta_key) FROM vce_users_meta"

### MySQL Connector Method

In [ ]:
cursor.execute(query)# Print contents of the cursor
for x in cursor:
  print(x)

### Pandas Method

In [ ]:
df = pd.read_sql(query, con = mydb)
print(df)